In [1]:
import torch
import torchvision

# Downloading MNIST dataset

In [2]:
batch_size_train = 64
batch_size_test = 1000

# Normalize values are the mean and standard deviation of MNIST. Mean = 0.1307, Stdev = 0.3081.
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                             torchvision.transforms.ToTensor(),
                             torchvision.transforms.Normalize(
                             (0.1307,), (0.3081,))])), 
                             batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                             torchvision.transforms.ToTensor(),
                             torchvision.transforms.Normalize(
                             (0.1307,), (0.3081,))])),
                             batch_size=batch_size_test, shuffle=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw




Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Hyperparemeters and more

In [26]:
n_epochs = 3
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

## Setting up network

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [28]:
coolestCnnEver = Net()
optimizer = optim.SGD(coolestCnnEver.parameters(), lr=learning_rate, momentum=momentum) # momentum to make it faster if needed.

In [29]:
# rerun this to reset loss arrays etc
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

## Define train procedure

In [8]:
def train(net, epoch, train_loader):
  net.train() # set model into training mode.

  for batch_idx, (data, labels) in enumerate(train_loader):
    optimizer.zero_grad()
    output = net(data)
    loss = F.nll_loss(output, labels)
    loss.backward()
    optimizer.step()

    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      

In [9]:
def test(net, test_loader):
  net.eval() # Set model into evaluation mode, no training.
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, labels in test_loader:
      output = net(data)
      test_loss += F.nll_loss(output, labels, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(labels.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

## Train dis shit

In [30]:
test(coolestCnnEver, test_loader)
for epoch in range(1, n_epochs + 1):
  train(coolestCnnEver, epoch, train_loader)
  test(coolestCnnEver, test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3037, Accuracy: 669/10000 (7%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.335748
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.300440
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.279269
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.271565
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.267453
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.249878
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.185932
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.196994
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.124859
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.096161
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.121677
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.014802
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.886009
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.832950
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.858891
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.546918
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.591930
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.353626
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.274130
Train Epoch: 1 [12160/6

## Training results
After 3 epochs we get 97% accuracy on the test set.

In [12]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [21]:
# in case we want to save the model
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%d%m%H%M_%S")
torch.save(coolestCnnEver, f'/content/gdrive/MyDrive/D7047E/CNN_MNIST/MNIST{current_time}')
#torch.save(optimizer.state_dict(), '/results/optimizer.pth') # NEED to also save this otherwise not trainable :(

# Transfer learning to SVHN
 ## Downloading SVHN dataset

In [13]:
from torchvision import transforms

transform = transforms.Compose([
                    transforms.CenterCrop(28),
                    transforms.Grayscale(num_output_channels=1),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5), (0.5))])

svhn_train_loader = torch.utils.data.DataLoader(torchvision.datasets.SVHN(root='/files/', split='train', download=True,  transform=transform),
                                                batch_size=batch_size_test, shuffle=True)
svhn_test_loader = torch.utils.data.DataLoader(torchvision.datasets.SVHN(root='/files/', split='test', download=True,  transform=transform),
                                               batch_size=batch_size_test, shuffle=True)

In [14]:
examples = enumerate(svhn_train_loader)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1000, 1, 28, 28])

In [24]:
mnistCNN = torch.load('/content/gdrive/MyDrive/D7047E/CNN_MNIST/MNIST23041453_02') # you need to change this to run it from your user
#mnistCNN.train() # Set to train mode for fine tuning.

In [31]:
test(coolestCnnEver, svhn_test_loader)
coolestCnnEver.train()
n_epochs = 50 # Will abort training if/when it peaks
for epoch in range(1, n_epochs + 1):
  train(coolestCnnEver, epoch, svhn_train_loader)
  test(coolestCnnEver, svhn_test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.1272, Accuracy: 7132/26032 (27%)

Train Epoch: 1 [0/73257 (0%)]	Loss: 2.219111
Train Epoch: 1 [10000/73257 (14%)]	Loss: 2.173585
Train Epoch: 1 [20000/73257 (27%)]	Loss: 2.140542
Train Epoch: 1 [30000/73257 (41%)]	Loss: 2.129264
Train Epoch: 1 [40000/73257 (54%)]	Loss: 2.092590
Train Epoch: 1 [50000/73257 (68%)]	Loss: 2.116441
Train Epoch: 1 [60000/73257 (81%)]	Loss: 2.078398
Train Epoch: 1 [70000/73257 (95%)]	Loss: 2.034437

Test set: Avg. loss: 1.9127, Accuracy: 10498/26032 (40%)

Train Epoch: 2 [0/73257 (0%)]	Loss: 2.062923
Train Epoch: 2 [10000/73257 (14%)]	Loss: 2.030369
Train Epoch: 2 [20000/73257 (27%)]	Loss: 1.996756
Train Epoch: 2 [30000/73257 (41%)]	Loss: 1.958107
Train Epoch: 2 [40000/73257 (54%)]	Loss: 1.996220
Train Epoch: 2 [50000/73257 (68%)]	Loss: 1.959916
Train Epoch: 2 [60000/73257 (81%)]	Loss: 1.944416
Train Epoch: 2 [70000/73257 (95%)]	Loss: 1.954834

Test set: Avg. loss: 1.7563, Accuracy: 12190/26032 (47%)

Train Epoch: 3 [0/73257 (0%)]	Loss

KeyboardInterrupt: ignored

After training the MnistCNN on SVHN dataset for 35 epochs we get an accuracy of 80%